In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.neighbors import KNeighborsRegressor
from sklearn import neighbors

from warnings import filterwarnings

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/Hitters.csv")
df.dropna(inplace=True)
dms = pd.get_dummies(df[["League","Division","NewLeague"]]) # One hot encoding
y = df["Salary"]
X_= df.drop(columns=["League","Division","NewLeague"],axis=1).astype(float)
X = pd.concat(objs=[X_,dms[["League_N","Division_W","NewLeague_N"]]],axis=1)
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=50)

In [ ]:
model = KNeighborsRegressor().fit(X_train, y_train)
dir(model)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_algorithm_metric',
 '_check_feature_names',
 '_check_n_features',
 '_estimator_type',
 '_fit',
 '_fit_X',
 '_fit_method',
 '_get_param_names',
 '_get_tags',
 '_kneighbors_reduce_func',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_tree',
 '_validate_data',
 '_validate_params',
 '_y',
 'algorithm',
 'effective_metric_',
 'effective_metric_params_',
 'feature_names_in_',
 'fit',
 'get_params',
 'kneighbors',
 'kneighbors_graph',
 'leaf_size',
 'metric',
 'metric_params

In [ ]:
print(model.algorithm)
print(model.n_neighbors)
print(model.radius)
print(model.get_params)

auto
5
None
<bound method BaseEstimator.get_params of KNeighborsRegressor()>


In [ ]:
model.metric

'minkowski'

In [ ]:
y_pred=model.predict(X_test)
np.sqrt(mean_squared_error(y_pred, y_test))

279.7128848364467

In [ ]:
#Model Tunning
#let's make predictions for different k neighbor values

RMSE =[]
for k in range(5, 10):
  model = KNeighborsRegressor(n_neighbors=k).fit(X_train, y_train) # p is 2 as default, euclidian
  y_pred = model.predict(X_test)
  RMSE.append(f"for {k} neighbor, RMSE: {np.sqrt(mean_squared_error(y_pred, y_test))}")
RMSE


['for 5 neighbor, RMSE: 279.7128848364467',
 'for 6 neighbor, RMSE: 285.2362846772276',
 'for 7 neighbor, RMSE: 295.2993243197654',
 'for 8 neighbor, RMSE: 305.3263377179035',
 'for 9 neighbor, RMSE: 316.1845276606918']

In [ ]:
model.p

In [ ]:
# Trying minkowski distance (p = 1)
RMSE =[]
for k in range(5, 10):
  model = KNeighborsRegressor(n_neighbors=k, p=1).fit(X_train, y_train)
  y_pred = model.predict(X_test)
  RMSE.append(f"for {k} neighbor, RMSE: {np.sqrt(mean_squared_error(y_pred, y_test))}")
RMSE

['for 5 neighbor, RMSE: 321.6268455627177',
 'for 6 neighbor, RMSE: 329.26469445329695',
 'for 7 neighbor, RMSE: 328.5749588523875',
 'for 8 neighbor, RMSE: 330.72735610031333',
 'for 9 neighbor, RMSE: 339.8454009171969']

In [ ]:
#GridSearchCV

In [ ]:
# Get the parameters as dictionary with their exact expression used KNeighborsRegressor()
knn_params = {"n_neighbors":np.arange(1,30,1), "p":np.array([1,2])}

# Create an empty model
model = KNeighborsRegressor()
# Determine the best parameters using GridSearch CV
knn_cv = GridSearchCV(estimator=model, param_grid=knn_params, cv=10)


In [ ]:
cv_model = knn_cv.fit(X_train, y_train)

In [ ]:
cv_model

GridSearchCV(cv=10, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
                         'p': array([1, 2])},
             return_train_score=True, verbose=-1)

In [ ]:
cv_model.best_params_

{'n_neighbors': 1, 'p': 2}

In [ ]:
# Final Model
knn_tuned = KNeighborsRegressor(n_neighbors=1, p=2).fit(X_train, y_train)
np.sqrt(mean_squared_error(y_test, knn_tuned.predict(X_test)))

217.09766950384335